In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("/Users/guilhermesilva/UFMG/Mestrado/Thesis/tinyc")

# `src.abstract_syntax_tree`

In [2]:
from src.abstract_syntax_tree import Node, AbstractSyntaxTree

## `Node`

In [3]:
node_with_set_value = Node(id=1, kind="TEST", value=1)
print(node_with_set_value)

ID: 1, Kind: TEST, Value: 1


In [4]:
node_with_default_value = Node(id=1, kind="TEST")
print(node_with_default_value)

ID: 1, Kind: TEST, Value: 0


## `AbstractSyntaxTree`

In [5]:
tree = AbstractSyntaxTree()
print(tree.root)

ID: 0, Kind: PROG, Value: 0


# `src.syntax_parser`

In [3]:
from src.syntax_parser import SyntaxParser

## `SyntaxParser`

In [4]:
parser = SyntaxParser()
vars(parser)

{'_global_id_manager': 1}

### `term`

In [5]:
test_id_term = parser.term("ID", 1)
print(test_id_term)

ID: 1, Kind: VAR, Value: 1


In [9]:
# This should raise an Exception, as `parenthesis_expression` is not yet implemented
test_other_term = parser.term("OTHER", 1)
print(test_other_term)

Could not run decorated function. The ID counter was not affected.
Could not run decorated function. The ID counter was not affected.


UnboundLocalError: local variable 'result' referenced before assignment

In [6]:
test_int_term = parser.term("INT", 1)
print(test_int_term)

ID: 2, Kind: CST, Value: 1


### `sum`

In [7]:
lhs = Node(id=-1, kind="INT", value=1)
rhs = Node(id=-2, kind="INT", value=2)

In [11]:
test_sum_plus = parser.sum(symbol="PLUS", lhs=lhs, rhs=rhs)
print(test_sum_plus)

for child in test_sum_plus.children:
    print(child)
    print(child.parent)

ID: 5, Kind: ADD, Value: -1
ID: -1, Kind: INT, Value: 1, Parent ID: 5
ID: 5, Kind: ADD, Value: -1
ID: -2, Kind: INT, Value: 2, Parent ID: 5
ID: 5, Kind: ADD, Value: -1


In [12]:
test_sum_minus = parser.sum(symbol="MINUS", lhs=lhs, rhs=rhs)
print(test_sum_minus)

for child in test_sum_minus.children:
    print(child)
    print(child.parent)

ID: 6, Kind: SUB, Value: -1
ID: -1, Kind: INT, Value: 1, Parent ID: 6
ID: 6, Kind: SUB, Value: -1
ID: -2, Kind: INT, Value: 2, Parent ID: 6
ID: 6, Kind: SUB, Value: -1


## `comparison`

In [9]:
lhs = Node(id=-1, kind="INT", value=1)
rhs = Node(id=-2, kind="INT", value=2)

In [10]:
test_comparison = parser.comparison(lhs=lhs, rhs=rhs)
print(test_comparison)

for child in test_comparison.children:
    print(child)
    print(child.parent)

ID: 4, Kind: LT, Value: -1
ID: -1, Kind: INT, Value: 1, Parent ID: 4
ID: 4, Kind: LT, Value: -1
ID: -2, Kind: INT, Value: 2, Parent ID: 4
ID: 4, Kind: LT, Value: -1
